# Notebook 5: Entropy and quantum information

In this notebook we'll have a closer look at entropy and the transmission of quantum information over a quantum circuit. 

We begin by trying to understand the von Neumann entropy and various related quantities.

In [ ]:
import numpy as np

# Import cirq, install it if it's not installed.
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

## Von Neumann entropy

Recall tha the von Neumann entropy of a state $\rho$ is defined by
$$
    S(\rho):= - \operatorname{tr} \rho \log \rho
$$

We can implement it in terms of the singular/eigenvalues $\lambda_i$ of $\rho$ (remember, $\rho$ is PSD, so the two notions coincide). This gives the formula
$$
    S(\rho) = - \sum_i \lambda_i \log \lambda_i
$$

### Exercise 1a
> Using `scipy.linalg.eigvalsh` to find the eigenvalues of a Hermitian matrix, define a method `entropy(rho)` that computes the entropy of a density matrix. Use `np.clip` to clip the eigenvalues to a small positive value to avoid computing the logarithm of 0 or a negative number.

In [ ]:
import scipy.linalg


def entropy(rho):
    # YOUR CODE HERE
    ...


# First test entropy on a pure state on 4 qubits.
pure_state = np.random.normal(size=2**4) + 1j * np.random.normal(size=2**4)
pure_state /= np.linalg.norm(pure_state)
rho_pure = np.outer(pure_state, pure_state.conj())
print(entropy(rho_pure))  # should be positive and very close to 0

# Next test it on the state I/d
rho = np.eye(2**4)
rho /= rho.trace()
print(entropy(rho))  # should be 4; if ~2.77 use a different logarithm


We know that the entropy of a state $\rho$ on $n$ qubits is between $0$ (a pure state) and $d$ (a uniform mixture of $d$ orthogonal pure states). Furthermore a unitary quantum circuit (i.e. without any noise) can never increase the entropy of a state. We know that it is impossible for any quantum operation to _decrease_ the entropy of a state.
Logically we therefore expect that the more noisy a circuit becomes, the more it will increase the entropy of a state.

Let's put these two notions to the test, by varying the level of noise in a quantum circuit. First we make a circuit, and simulate it in the absence of noise.

In [ ]:
qubits = cirq.LineQubit.range(4)
circuit = cirq.Circuit()
circuit.append([cirq.H(qubits[0]), cirq.H(qubits[2])])
circuit.append(
    [cirq.CNOT(qubits[0], qubits[1]), cirq.CNOT(qubits[2], qubits[3])]
)
circuit.append(
    [cirq.CNOT(qubits[0], qubits[2]), cirq.CNOT(qubits[1], qubits[3])]
)


print(circuit)

simulator = cirq.Simulator()
simulator.simulate(circuit)


We can now make a noisy version of the circuit by using `circuit.with_noise`. For example, we create a circuit that add a depolarization noise at every moment in the circuit. 

In [ ]:
noisy_circuit = circuit.with_noise(cirq.depolarize(0.01))
print(noisy_circuit)
simulator = cirq.DensityMatrixSimulator()
rho = simulator.simulate(noisy_circuit).final_density_matrix
cirq.qis.von_neumann_entropy(rho)  # equivalent to entropy(rho)


### Exercise 1b
> Make a noisy version of the quantum circuit, adding a `bit_flip(p)` noise at every moment in the circuit, and simulate the final density matrix of this circuit. Plot the relation between `p` and the entropy of the final density matrix.

In [ ]:
import matplotlib.pyplot as plt

simulator = cirq.DensityMatrixSimulator()
noise_prob = np.linspace(0, 1, 100)
entropies = []
for p in noise_prob:
    # YOUR CODE HERE
    ...

plt.plot(noise_prob, entropies)


## Operator sum representation and the Quantum Fano inequality

Recall that any quantum operation can be modelled by a unitary operation in a larger system. For example, the bit flip channel with probability $p<0.5$ is equivalent to the following circuit:

In [ ]:
qubit, environment = cirq.LineQubit.range(2)
circuit = cirq.Circuit()
circuit.append(cirq.H(environment))
circuit.append(cirq.CNOT(environment, qubit))
circuit


For a more general bit flip gate, we first define the unitary matrix $A(p)$ by
$$
    A = \begin{pmatrix}\sqrt{1-p}&-\sqrt{p}\\ \sqrt{p}&\sqrt{1-p}\end{pmatrix}
$$
And then replace $H$ above by $A$. We do this by implementing a custom 1-qubit gate below.

In [ ]:
class BitFlipUnitary(cirq.Gate):
    def __init__(self, p):
        self.p = p
        super().__init__()

    def _num_qubits_(self):
        return 1

    def _unitary_(self):
        A = np.array(
            [[np.sqrt(1 - p), -np.sqrt(p)], [np.sqrt(p), np.sqrt(1 - p)]]
        )
        return A

    def _circuit_diagram_info_(self, args):
        return f"A({self.p})"


p = 0.1
circuit = cirq.Circuit()
circuit.append(BitFlipUnitary(p)(environment))
circuit.append(cirq.CNOT(environment, qubit))
circuit


This unitary representation is useful, because it allows us to extract the _operation elements_ ${E_i}$ of the associated quantum operation $\mathcal E$ relatively easily. If $|0\rangle,\,|1\rangle$ is the basis of the working qubit above, and $U$ denotes the unitary matrix of the circuit, we have:
$$
    E_i = \langle i| U | 0\rangle
$$
In our case $U$ is a 4x4 matrix, so how do we obtain the operator elements from it? 
One trick we can do is to first reshape it to a (2x2)x(2x2) _tensor_ `U_tens`. This has 4 indices, `U_tens[i1,i2,i3,i3]`. The indices `i1` and `i3` correspond to the _first_ qubit, whereas the indices `i2`, `i4` correspond to the second (environment) qubit. Therefore in this context, `E0 = U_tens[:, 0, :, 0]`. That is, we fix the working qubit to the state $|0\rangle$ in both the input and output of the operator. Similarly `E1 = U_tens[:, 1, :, 0]`.

In [ ]:
U = circuit.unitary()
U_tens = U.reshape(2, 2, 2, 2)
E0 = U_tens[:, 0, :, 0]
E1 = U_tens[:, 1, :, 0]
E0, E1


### Exercise 2a
> Using the operation elements, we can write 
$$
    \mathcal E(\rho) = \sum_i E_i^\dagger \rho E_i
$$
> Use the formula above to verify that `E0` and `E1` are indeed the operation elements of the `p=0.1` bit-flip channel. Do this by computing the formula above for a random 1-qubit state `rho`. Then compare this to the final density matrix obtained after applying a circuit to `rho` consisting of a single `cirq.bit_flip(0.1)` gate.

In [ ]:
def random_state(n):
    shape = (2**n, 2**n)
    rho = np.random.normal(size=shape) + 1j * np.random.normal(size=shape)
    rho = rho @ rho.conj().T
    rho = rho / rho.trace()
    return rho.astype(np.complex64)


rho = random_state(1)

# YOUR CODE HERE


The operator sum representation is very useful for computing quite a few things. For example, it allows us to compute the _entanglement fidelity_ (eq. 9.135):
$$
    F(\rho,\mathcal E) = \sum_i|\operatorname{tr}(\rho E_i)|^2
$$
This measures how much the operation $\mathcal E$ changes the entanglement between the qubit of interest and the environment. 

A second quantity of interest is the _entropy exchange_ of $\mathcal E$ for the state $\rho$, which ise defined by eq. (12.111):
$$
    S(\rho,\mathcal E) = -\operatorname{tr} (W \log W)
$$
where $W$ is a matrix with entries
$$
    W_{ij} = \operatorname{E_i\rho E_j^\dagger}
$$

### Exercise 2b
> Define a functions `entanglement_fidelity` and `entropy_exchange` which respectively compute $F(\rho,\mathcal E)$ and $S(\rho,\mathcal E)$ for a given state $\rho$ and quantum operation $\mathcal E$ given as a list of operator elements.

In [ ]:
def entanglement_fidelity(rho, operator_elements):
    # YOUR CODE HERE
    ...


def entropy_exchange(rho, operator_elements):
    W_shape = (len(operator_elements), len(operator_elements))
    W = np.zeros(W_shape, dtype=complex)
    # YOUR CODE HERE
    # Form the matrix W using 2 for loops and compute its entropy


print("rho is pure state |0>")
rho = np.array([[1, 0], [0, 0]])
print(entanglement_fidelity(rho, [E0, E1]))  # should be 0.9
print(entropy_exchange(rho, [E0, E1]))  # should be around 0.469 = H(0.1)
print(-0.1 * np.log2(0.1) - 0.9 * np.log2(0.9))


print("\nrho is pure state |0>/sqrt(2) + |1>/sqrt(2)")
# This state is invariant under bit flips, and hence the circuit
# leaves this state invariant
rho = np.array([[1, 1], [1, 1]]) / 2
print(entanglement_fidelity(rho, [E0, E1]))  # should be 1
print(entropy_exchange(rho, [E0, E1]))  # should be 0


Next we consider the Quantum Fano inequality (Theorem 12.9), which states that
$$
    S(\rho,\mathcal E)\leq H(F(\rho,\mathcal E)) + (1-F(\rho,\mathcal E))\log (d^2-1)
$$
where $H$ is the binary Shannon entropy, and $d$ is the dimension of the system. In our case $d=2$, 

### Exercise 2c
> Plot the left-hand side and right-hand side of the Fano inequality for the bit-flip gate for values of $p$ between 0 and 1. First write a function that returns the operator elements of the bit-flip gate for specified $p$. All of this is done for a fixed random state $\rho$.

In [ ]:
def bit_flip_operator_elements(p):
    """Returns the operator elements of the BitFlip(p) gate."""
    # YOUR CODE HERE


def binary_shannon_entropy(p):
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)


rho = random_state(1)
probs = np.linspace(1e-5, 1, 100)
lhs = []
rhs = []
for p in probs:
    # YOUR CODE HERE
    ...

plt.plot(probs, lhs, label="Entropy exchange")
plt.plot(probs, rhs, label="Quantum Fano bound")
plt.legend()
